### Main: GUI Backend

In [1]:
import gradio as gr
from pathlib import Path

from data_setup import read_csvs_from_directory, create_sqlite_db_from_dfs, get_metadata_from_dfs
from llm_handler import configure_llm, generate_sql_query
from query_executor import run_sql_query

In [2]:
# Main function that wraps your logic
def generate_insights(dataset_name, api_key, question):
    dataframes = read_csvs_from_directory(f"Datasets/{dataset_name}")
    conn = create_sqlite_db_from_dfs(dataframes)
    metadata = get_metadata_from_dfs(dataframes)
    model = configure_llm(api_key)
    
    sql = generate_sql_query(question, metadata, model)
    result = run_sql_query(sql, conn)
    
    return sql, result

# Dataset options from folder
dataset_names = [f.name for f in Path("Datasets").iterdir() if f.is_dir()]

# Define the interface
with gr.Blocks(title="LLM Data Analysis") as demo:
    gr.Markdown("# LLM-powered Data Insights")
    
    with gr.Row():
        dataset_dropdown = gr.Dropdown(choices=dataset_names, label="Select Local Dataset")
        api_input = gr.Textbox(label="Gemini API Key", type="password")

    question_input = gr.Textbox(lines=4, label="Ask a question in natural language")

    run_button = gr.Button("Generate Insights")

    sql_output = gr.Textbox(label="Generated SQL Query", lines=3, interactive=False)
    result_output = gr.Dataframe(label="Query Result")

    run_button.click(
        fn=generate_insights,
        inputs=[dataset_dropdown, api_input, question_input],
        outputs=[sql_output, result_output]
    )

# Launch in new browser tab
demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [3]:
# !jupyter nbconvert --to script data_setup.ipynb
# !jupyter nbconvert --to script llm_handler.ipynb
# !jupyter nbconvert --to script query_executor.ipynb